## Initial Setting

In [2]:
import numpy as np
import pandas as pd
import msm
import sklearn.metrics
from simulatedata import simulatedata
from simulatedata_initial_state import simulatedata_with_initial_prob

ModuleNotFoundError: No module named 'simulatedata'

In [6]:
import numpy as np
import pandas as pd
import scipy.optimize as opt
from scipy import stats
from statsmodels import regression
import statsmodels.formula.api as sm
from numba import jit, njit, prange, float64, int64
import os
import glob
from datetime import datetime, timedelta, date
import scipy.stats as stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

##utiles pour GARCH
from random import gauss
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from arch.__future__ import reindexing


## Data Selection

In [4]:
dat2 = pd.read_csv("DEXJPUS_calvet.csv")                                   
dat2 = dat2.loc[dat2.DEXJPUS != "."].DEXJPUS.astype(float)
dat2 = np.array(dat2)
dat2_rtn = dat2[0:-1]
dat2 = np.log(dat2[1:])-np.log(dat2[0:-1])
dat2 = dat2[dat2 != 0]
dat2 = dat2[:,np.newaxis]
dat2_rtn = dat2_rtn[:,np.newaxis]
T = len(dat2)
E = np.rint(0.6*T).astype(int)
dat2E = dat2[0:E,]
dat2F = dat2[E:,]
dat2_rtnE = dat2_rtn[0:E,]
dat2_rtnF = dat2_rtn[E:,]
dat2E.shape, dat2F.shape

((4252, 1), (2835, 1))

## Estimate Parameter

In [19]:
data = dat2E               # Simulated dta
kbar = 4
niter = 1
temperature = 1.0
stepsize = 1.0

parameters, LL, niter, output = msm.glo_min(kbar, data, niter, temperature, stepsize)

# name parameters for later use:
b_sim = parameters[0]
m_0_sim = parameters[1]
gamma_kbar_sim = parameters[2]
sigma_sim = parameters[3]
LL_sim = LL

print("Parameters from glo_min for Simulated dataset: ", "\n"
      "kbar = ", kbar,"\n"
      'b = %.5f' % b_sim,"\n"
      'm_0 = %.5f' % m_0_sim,"\n"
      'gamma_kbar = %.5f' % gamma_kbar_sim,"\n"
      'sigma = %.5f' % (sigma_sim*np.sqrt(252)),"\n"
      'Likelihood = %.5f' % LL_sim,"\n"
      "niter = " , niter,"\n"
      "output = " , output,"\n")

NameError: name 'msm' is not defined

In [21]:
b = parameters[0]
m0 = parameters[1]
gamma_kbar = parameters[2]
sigma = parameters[3]

theta_in = [b, gamma_kbar, sigma]

NameError: name 'parameters' is not defined

## Analysis Garch(2,2)

In [48]:
def simulategarchdata(T, omega, alpha1,alpha2, beta1, beta2):
    n = T

    test_size = int(n*0.1)

    series = [gauss(0,1), gauss(0,1)]
    vols = [1, 1]

    for _ in range(n-2):
        new_vol = np.sqrt(omega + alpha1*series[-1]**2 + alpha2*series[-2]**2 + beta1*vols[-1]**2 + beta2*vols[-2]**2)
        new_val = gauss(0,1) * new_vol

        vols.append(new_vol)
        series.append(new_val)
    return np.array([vols,series])

## rt est donné par simulateagarchdata[10]

### in sample

In [15]:
delta = 10
train, test = dat2E[:-delta], dat2E[-delta:]
model = arch_model(train, p=1, q=1, rescale=False)
model_fit = model.fit()
model_fit.summary()

Iteration:      1,   Func. Count:      6,   Neg. LLF: 470928971739.7442
Iteration:      2,   Func. Count:     20,   Neg. LLF: 35708607.506961875
Optimization terminated successfully    (Exit mode 0)
            Current function value: -15924.709313884381
            Iterations: 2
            Function evaluations: 29
            Gradient evaluations: 2


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                15924.7
Distribution:                  Normal   AIC:                          -31841.4
Method:            Maximum Likelihood   BIC:                          -31816.0
                                        No. Observations:                 4242
Date:                Wed, Apr 06 2022   Df Residuals:                     4241
Time:                        09:54:32   Df Model:                            1
                                   Mean Model                                  
===============================================================================
                  coef    std err          t      P>|t|        95.0% Conf. Int.
-------------------------------------------------------------------------------
mu         -5.9418e-05  6.041e-06     -9.837  7.836e-23 [-7.126e-05,-4.758e-05]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega      7.8368e-07  5.373e-12  1.458e+05      0.000 [7.837e-07,7.837e-07]
alpha[1]       0.1000  1.625e-02      6.154  7.570e-10   [6.815e-02,  0.132]
beta[1]        0.8800  1.583e-02     55.594      0.000     [  0.849,  0.911]
============================================================================

Covariance estimator: robust
"""

In [57]:
x = dat2E[:-delta,]
y = dat2E[delta:,]

y_pred = simulategarchdata(len(y), 7.8e-7,0.1,0,0.88,0)
print(y_pred[1])
print((y))

R2 = sklearn.metrics.r2_score(y**2,y_pred[1]**2)
MSE = sklearn.metrics.mean_squared_error(y**2,y_pred[1]**2)
R2,MSE

[ 1.37901363e-01 -8.28103515e-01  1.07320090e+00 ... -4.44235438e-04
  4.72526371e-03 -5.54044611e-03]
[[ 0.00052808]
 [-0.00764663]
 [ 0.00791057]
 ...
 [-0.00140834]
 [ 0.0045309 ]
 [ 0.0060611 ]]


(-1944218.4645521068, 0.025048445600397366)

## Analysis MSM Binomial

### in sample

In [ ]:
delta = 10
x = dat2E[:-delta,]
y = dat2E[delta:,]

y_pred = simulatedata(b,m0,gamma_kbar,sigma,kbar,E-delta)

R2 = sklearn.metrics.r2_score(y**2,y_pred**2)
MSE = sklearn.metrics.mean_squared_error(y**2,y_pred**2)
R2,MSE

### out of sample

In [ ]:
delta = 10
x = dat2F[:-delta,]
y = dat2F[delta:,]
y_pred = simulatedata(b,m0,gamma_kbar,sigma,kbar,T-E-delta)
R2 = sklearn.metrics.r2_score(y**2,y_pred**2)
MSE = sklearn.metrics.mean_squared_error(y**2,y_pred**2)
R2,MSE

## Analysis MSM initial state

### in sample

In [ ]:
delta = 1
x = dat2E[:-delta,]
y = dat2E[delta:,]

y_pred = np.zeros((E-delta,1))
for i in range(E-delta):
    prob=msm.g_pi_t(m0, kbar, x[:i+1,], theta_in)
    y_pred[i,0] = simulatedata_with_initial_prob(b,m0,gamma_kbar,sigma,kbar,prob,1)[0]

R2 = sklearn.metrics.r2_score(y**2,y_pred**2)
MSE = sklearn.metrics.mean_squared_error(y**2,y_pred**2)
R2,MSE

### out of sample

In [ ]:
delta = 1
x = dat2F[:-delta,]
y = dat2F[delta:,]

y_pred = np.zeros((T-E-delta,1))
for i in range(T-E-delta):
    prob=msm.g_pi_t(m0, kbar, x[:i+1,], theta_in)
    y_pred[i,0] = simulatedata_with_initial_prob(b,m0,gamma_kbar,sigma,kbar,prob,1)[0]

R2 = sklearn.metrics.r2_score(y**2,y_pred**2)
MSE = sklearn.metrics.mean_squared_error(y**2,y_pred**2)
R2,MSE